# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv('ADBL_main.csv')
df.head()

,Symbol,High,Low,Close,Date
0,ADBL,499.0,488.0,492.0,01_01_2022
1,ADBL,412.0,400.1,411.0,01_02_2022
2,ADBL,418.0,412.0,415.0,01_03_2022
3,ADBL,424.0,410.0,422.0,01_04_2022
4,ADBL,430.0,418.0,419.0,01_05_2022


In [3]:
closedf = df.reset_index()['Close']

# Dropping Null values from data

In [4]:
closedf = closedf.dropna()
closedf

0      492.0
1      411.0
2      415.0
3      422.0
4      419.0
       ...  
250    314.0
251    314.0
252    305.1
253    307.0
254    305.0
Name: Close, Length: 255, dtype: float64

In [5]:
scaler = MinMaxScaler(feature_range=(0,1))
df1 = scaler.fit_transform(np.array(closedf).reshape(-1,1))
df1

array([[7.67967146e-01],
       [4.35318275e-01],
       [4.51745380e-01],
       [4.80492813e-01],
       [4.68172485e-01],
       [4.76386037e-01],
       [4.76386037e-01],
       [4.76386037e-01],
       [5.17453799e-01],
       [5.09240246e-01],
       [5.08829569e-01],
       [4.88706366e-01],
       [5.00616016e-01],
       [5.00616016e-01],
       [5.00616016e-01],
       [5.33880903e-01],
       [5.58521561e-01],
       [5.54414784e-01],
       [5.25667351e-01],
       [5.25667351e-01],
       [5.25667351e-01],
       [5.25667351e-01],
       [5.14168378e-01],
       [4.80492813e-01],
       [4.71868583e-01],
       [4.57494867e-01],
       [4.59958932e-01],
       [4.59958932e-01],
       [4.59958932e-01],
       [4.91581109e-01],
       [4.80492813e-01],
       [4.84599589e-01],
       [4.84599589e-01],
       [4.74743326e-01],
       [4.74743326e-01],
       [4.74743326e-01],
       [4.64065708e-01],
       [4.59958932e-01],
       [4.59958932e-01],
       [4.53798768e-01],


# Splitting dataset into train and test split

In [6]:
split_ratio = 0.7
train_size=int(len(df1)*split_ratio)
test_size=len(df1)-train_size
train_data, test_data = df1[0:train_size,:], df1[train_size:len(df1),:]

# Convert the data into a sequence of past time steps and future time steps

In [7]:
def create_dataset(data, time_steps=1):
    x, y = [], []
    for i in range(len(data) - time_steps - 1):
        a = data[i:(i + time_steps), 0]
        x.append(a)
        y.append(data[i + time_steps, 0])
    return np.array(x), np.array(y)

# Create Dataset

In [8]:
time_steps = 60
x_train, y_train = create_dataset(train_data, time_steps)
x_test, y_test = create_dataset(test_data, time_steps)

# Reshape the data for the LSTM model

In [9]:
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

# Build the LSTM model

In [10]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(1,time_steps)))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 128)            96768     
                                                                 
 lstm_1 (LSTM)               (None, 1, 128)            131584    
                                                                 
 lstm_2 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 360,065
Trainable params: 360,065
Non-trainable params: 0
_________________________________________________________________


# Train the model

In [11]:
history1 = model.fit(x_train, y_train, epochs=100, verbose=1, validation_data=(x_test, y_test))
history1

Epoch 1/100
4/4 [==============================] - 6s 345ms/step - loss: 0.0699 - accuracy: 0.0085 - val_loss: 0.2024 - val_accuracy: 0.0000e+00
Epoch 2/100
4/4 [==============================] - 0s 15ms/step - loss: 0.0634 - accuracy: 0.0085 - val_loss: 0.1932 - val_accuracy: 0.0000e+00
Epoch 3/100
4/4 [==============================] - 0s 14ms/step - loss: 0.0524 - accuracy: 0.0085 - val_loss: 0.1782 - val_accuracy: 0.0000e+00
Epoch 4/100
4/4 [==============================] - 0s 15ms/step - loss: 0.0352 - accuracy: 0.0085 - val_loss: 0.1538 - val_accuracy: 0.0000e+00
Epoch 5/100
4/4 [==============================] - 0s 15ms/step - loss: 0.0196 - accuracy: 0.0085 - val_loss: 0.1209 - val_accuracy: 0.0000e+00
Epoch 6/100
4/4 [==============================] - 0s 13ms/step - loss: 0.0195 - accuracy: 0.0085 - val_loss: 0.1038 - val_accuracy: 0.0000e+00
Epoch 7/100
4/4 [==============================] - 0s 14ms/step - loss: 0.0197 - accuracy: 0.0085 - val_loss: 0.1072 - val_accuracy: 0.

In [12]:
# Test the model
prediction_test = model.predict(x_test)
prediction_test = scaler.inverse_transform(prediction_test)

1/1 [==============================] - 1s 990ms/step


# Calculate Accuracy

In [13]:
#calculate accuracy
print('Accuracy: %.2f' % (model.evaluate(x_train, y_train)[1]*100))

4/4 [==============================] - 0s 3ms/step - loss: 6.6818e-04 - accuracy: 0.0085
Accuracy: 0.85


# Calculate the root mean squared error

In [14]:
y_inverse = scaler.inverse_transform(y_test.reshape(-1,1))
rmse_test = np.sqrt(np.mean(((prediction_test - y_inverse) ** 2)))
print('RMSE:', rmse_test)

RMSE: 11.436213793286539


# Plotting Model

In [16]:
# Get the most recent 60 days of stock price data
recent_data = df1[-60:]

# Reshape the data for the LSTM model
x_recent = recent_data.reshape(1, 1, 60)

# Predict the stock prices for the next day and append to the input data
predicted_price = model.predict(x_recent)
df1 = np.append(df1, predicted_price)

# Shift the input data by one day
for i in range(6):
    recent_data = df1[-60:]
    x_recent = recent_data.reshape(1, 1, 60)
    predicted_price = model.predict(x_recent)
    df1 = np.append(df1, predicted_price)
    
# Inverse transform the predicted prices
predicted_prices = scaler.inverse_transform(df1[-7:].reshape(-1, 1))

# Print the predicted prices for the next 7 days
df_predicted = pd.DataFrame(predicted_prices, columns=['Predicted Prices'])
df_predicted.to_csv('predicted_prices.csv', index=False)

1/1 [==============================] - 0s 20ms/step


# Checking loss in model